# 02 - QAOA Demo (MaxCut, tiny graph)

In [ ]:

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator
import numpy as np

edges = [(0,1),(1,2),(2,0)]
n=3
H = SparsePauliOp.from_list([("III",0.0)])
for (i,j) in edges:
    s = ["I"]*n; s[i] = "Z"; s[j] = "Z"
    H += 0.5*SparsePauliOp.from_list([("".join(s), -1.0)])
H += 0.5*len(edges)*SparsePauliOp.from_list([("I"*n, 1.0)])

def qaoa_circuit(gamma, beta):
    qc = QuantumCircuit(n)
    for q in range(n): qc.h(q)
    for (i,j) in edges:
        qc.cx(i,j); qc.rz(2*gamma, j); qc.cx(i,j)
    for q in range(n): qc.rx(2*beta, q)
    return qc

est = Estimator()

def exp_cost(gamma, beta):
    return float(est.run(qaoa_circuit(gamma, beta), H).result().values[0])

grid = np.linspace(0, np.pi, 21)
best_val, best_params = 1e9, None
for g in grid:
    for b in grid:
        val = exp_cost(g,b)
        if val < best_val:
            best_val, best_params = val, (g,b)
best_params, best_val
